# Notebook 3 - Orthorectify GOES data using goes-ortho

This tests Steven's orthorectification code from goes-ortho. Copied from `goes-ortho/docs/examples
/goes-orthorectify.ipynb` on github.

The input is raw GOES ABI netcdf files and the output is corrected netcdfs. These can then be converted to zarr and processed into RGB composites as I have done with previous code (see `/processing/00_goes_rad_corrections.ipynb`)

In [1]:
import goes_ortho
import orthorectify_modded
import xarray as xr
import matplotlib.pyplot as plt
import os

import warnings
warnings.filterwarnings("ignore")

We will use the goes_ortho.ortho() function to orthorectify and clip a GOES ABI image for an area within lat/lon bounds we'll specify.

First, provide a filepath to the GOES ABI image we want to orthorectify:

In [2]:
path = '/storage/cdalden/goes/washington/goes17/2022/8/4/ABI-L1b-RadC/20/C13/'
goes_image_path = path + "OR_ABI-L1b-RadC-M6C13_G17_s20222162056177_e20222162058561_c20222162059001.nc"

We can specify a list of the variables in the dataset that we want to include in the orthorectified dataset:

In [3]:
data_vars = ["Rad"]

And then we need to provide a new filepath and filename for where to save the final orthorectified dataset:

In [4]:
new_goes_filename = path + "OR_ABI-L1b-RadC-M6C13_G16_s20222161601175_e20222161603559_c20222161604015_ortho.nc"

Finally, specify the lat/lon bounds for the area we want to orthorectify, like (min lon, min lat, max lon, max lat):

In [5]:
bounds = (-124, 48, -121, 49)

Run the function!

Note: goes_ortho.ortho does not exist as described in the example notebook, using goes_ortho.orthorectify.ortho instead

In [6]:
api_key = "41d14aae7e761c0de3e8f99aa4fd24d9"

orthorectify_modded.ortho(
    goes_image_path,
    data_vars,
    bounds,
    api_key,
    new_goes_filename,
    dem_filepath=None,
    demtype="SRTMGL3",
    keep_dem=False,
)

https://portal.opentopography.org/API/globaldem?demtype=SRTMGL3&west=-124&south=48&east=-121&north=49&outputFormat=GTiff&API_Key=41d14aae7e761c0de3e8f99aa4fd24d9
temp_SRTMGL3_DEM.tif


In [7]:
new_goes_filename

'/storage/cdalden/goes/washington/goes17/2022/8/4/ABI-L1b-RadC/20/C13/OR_ABI-L1b-RadC-M6C13_G16_s20222161601175_e20222161603559_c20222161604015_ortho.nc'

Open the new file we created, and plot it:

In [8]:
ds_ortho = xr.open_dataset(new_goes_filename)
ds = xr.open_dataset(goes_image_path)
ds_ortho

<xarray.Dataset> Size: 225MB
Dimensions:           (latitude: 1200, longitude: 3600)
Coordinates:
    t                 datetime64[ns] 8B ...
  * longitude         (longitude) float64 29kB -124.0 -124.0 ... -121.0 -121.0
  * latitude          (latitude) float64 10kB 49.0 49.0 49.0 ... 48.0 48.0 48.0
    dem_px_angle_x    (latitude, longitude) float64 35MB ...
    dem_px_angle_y    (latitude, longitude) float64 35MB ...
Data variables:
    elevation         (latitude, longitude) float32 17MB ...
    Rad               (latitude, longitude) float32 17MB ...
    tb                (latitude, longitude) float32 17MB ...
    abi_fixed_grid_x  (latitude, longitude) float64 35MB ...
    abi_fixed_grid_y  (latitude, longitude) float64 35MB ...
    zone_labels       (latitude, longitude) int64 35MB ...
Attributes: (12/21)
    longitude_of_projection_origin:       -137.0
    semi_major_axis:                      6378137.0
    semi_minor_axis:                      6356752.31414
    satellite_height:                     42164160.0
    grs80_eccentricity:                   0.0818191910435
    longitude_of_projection_origin_info:  longitude of geostationary satellit...
    ...                                   ...
    dem_ifov_info:                        instantaneous field of view (angula...
    dem_px_angle_x_info:                  DEM grid cell X coordinate (east/we...
    dem_px_angle_y_info:                  DEM grid cell Y coordinate (north/s...
    longitude_info:                       longitude from DEM file
    latitude_info:                        latitude from DEM file
    elevation_info:                       elevation from DEM file

In [9]:
ds

<xarray.Dataset> Size: 63kB
Dimensions:                                           (y: 63, x: 121,
                                                       number_of_time_bounds: 2,
                                                       number_of_image_bounds: 2,
                                                       num_star_looks: 24,
                                                       band: 1)
Coordinates:
    t                                                 datetime64[ns] 8B ...
  * y                                                 (y) float64 504B 0.1251...
  * x                                                 (x) float64 968B 0.0246...
    y_image                                           float32 4B ...
    x_image                                           float32 4B ...
    band_id                                           (band) int8 1B ...
    band_wavelength                                   (band) float32 4B ...
    t_star_look                                       (num_star_looks) datetime64[ns] 192B ...
    band_wavelength_star_look                         (num_star_looks) float32 96B ...
Dimensions without coordinates: number_of_time_bounds, number_of_image_bounds,
                                num_star_looks, band
Data variables: (12/37)
    Rad                                               (y, x) float32 30kB ...
    DQF                                               (y, x) float32 30kB ...
    time_bounds                                       (number_of_time_bounds) datetime64[ns] 16B ...
    goes_imager_projection                            int32 4B ...
    y_image_bounds                                    (number_of_image_bounds) float32 8B ...
    x_image_bounds                                    (number_of_image_bounds) float32 8B ...
    ...                                                ...
    algorithm_dynamic_input_data_container            int32 4B ...
    processing_parm_version_container                 int32 4B ...
    algorithm_product_version_container               int32 4B ...
    star_id                                           (num_star_looks) float32 96B ...
    channel_integration_time                          float64 8B ...
    channel_gain_field                                float64 8B ...
Attributes: (12/30)
    naming_authority:          gov.nesdis.noaa
    Conventions:               CF-1.7
    standard_name_vocabulary:  CF Standard Name Table (v35, 20 July 2016)
    institution:               DOC/NOAA/NESDIS > U.S. Department of Commerce,...
    project:                   GOES
    production_site:           WCDAS
    ...                        ...
    timeline_id:               ABI Mode 6
    date_created:              2022-08-04T20:59:00.1Z
    time_coverage_start:       2022-08-04T20:56:17.7Z
    time_coverage_end:         2022-08-04T20:58:56.1Z
    LUT_Filenames:             SpaceLookParams(FM2A_CDRL79RevP_PR_09_08_04)-6...
    id:                        d23be8f6-1bc4-49fc-a7f8-7b83055bfdc0

## Combine all previous work so that we can processing 1 day of data with minimal functions

1. run ortho script for all files in each day
    - this works but its really slow, need to parallize this (15s/file so potentially 3 hrs for 1 day)
2. combine all files for each day into 1 zarr per channel
3. convert to RGB values and create a zarr for all channels for each day

### Step 1 - Orthorectify raw GOES files

In [16]:
# Define the root directory to start searching
root_dir = '/storage/cdalden/goes/washington/goes17/2022/8/4/ABI-L1b-RadC/'

# Loop through all subdirectories and files
for subdir, _, files in os.walk(root_dir):
    for file in files:
        if file.endswith('.nc'):  # Check if the file is a NetCDF file
            netcdf_path = os.path.join(subdir, file)
            print('working on... ' + netcdf_path)

            # define all args needed for the ortho function
            goes_image_path = netcdf_path
            data_vars = ["Rad"]
            new_goes_filename = netcdf_path.replace('.nc', '_ortho.nc')
            bounds = (-124, 48, -121, 49)
            api_key = "41d14aae7e761c0de3e8f99aa4fd24d9"

            orthorectify_modded.ortho(
                goes_image_path,
                data_vars,
                bounds,
                api_key,
                new_goes_filename,
                dem_filepath=None,
                demtype="SRTMGL3",
                keep_dem=False,
            )

            # Delete the original NetCDF file
            os.remove(netcdf_path)

working on/storage/cdalden/goes/washington/goes17/2022/8/3/ABI-L1b-RadC/00/C02/OR_ABI-L1b-RadC-M6C02_G17_s20222150001177_e20222150003550_c20222150003570.nc


RasterioIOError: 'temp_SRTMGL3_DEM.tif' not recognized as being in a supported file format.